<a href="https://colab.research.google.com/github/JaydaBubel/lab-web-scraping-multiple-pages/blob/master/Lab_Webscraping_multiple_JaydaB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [42]:
r = requests.get('https://en.wikipedia.org/wiki/Lists_of_songs')
r.status_code

200

In [43]:
r.headers

{'date': 'Thu, 14 Sep 2023 07:33:07 GMT', 'server': 'mw1387.eqiad.wmnet', 'x-content-type-options': 'nosniff', 'content-language': 'en', 'x-frame-options': 'DENY', 'accept-ch': '', 'vary': 'Accept-Encoding,Cookie', 'last-modified': 'Thu, 14 Sep 2023 01:26:27 GMT', 'content-type': 'text/html; charset=UTF-8', 'content-encoding': 'gzip', 'age': '42359', 'x-cache': 'cp1077 hit, cp1079 hit/25', 'x-cache-status': 'hit-front', 'server-timing': 'cache;desc="hit-front", host;desc="cp1079"', 'strict-transport-security': 'max-age=106384710; includeSubDomains; preload', 'report-to': '{ "group": "wm_nel", "max_age": 604800, "endpoints": [{ "url": "https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" }] }', 'nel': '{ "report_to": "wm_nel", "max_age": 604800, "failure_fraction": 0.05, "success_fraction": 0.0}', 'set-cookie': 'WMF-Last-Access=14-Sep-2023;Path=/;HttpOnly;secure;Expires=Mon, 16 Oct 2023 12:00:00 GMT, WMF-L

In [57]:
#parse and get lists of links for rock band songs
r = requests.get('https://en.wikipedia.org/wiki/Lists_of_songs')
soup = BeautifulSoup(r.content, 'html.parser')

def filter_function(tag):
    return tag.has_attr('title') and tag['title'].startswith("List of songs in Rock")

links = soup.find_all(filter_function)

base_url = "https://en.wikipedia.org"

links_list = [base_url + link['href'] for link in links]

links_list

['https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band',
 'https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band_2',
 'https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band_3',
 'https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band_4',
 'https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band_Blitz',
 'https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band_Unplugged']

In [73]:
#scrape song, artist and year
def scrape_songs_artists_years_from_url(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
#find table
    table = soup.find('table', class_='wikitable')
#make lists
    songs = []
    artists = []
    years = []

    if table:
        rows = table.find_all('tr')[1:]

        for row in rows:
            cells = row.find_all('td')

#check if there are at least three cells in the row
            if len(cells) >= 3:
#extract song, artist, and year from correct cells
                song_title = cells[0].get_text(strip=True)
                artist_name = cells[1].get_text(strip=True)
                year = cells[2].get_text(strip=True)

#append to lists
                songs.append(song_title)
                artists.append(artist_name)
                years.append(year)

    return songs, artists, years

In [74]:
#List of URLs
urls = [
    'https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band',
    'https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band_2',
    'https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band_3',
    'https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band_4',
    'https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band_Blitz',
    'https://en.wikipedia.org/wiki/List_of_songs_in_Rock_Band_Unplugged'
]

all_songs = []
all_artists = []
all_years = []

for url in urls:
    songs, artists, years = scrape_songs_artists_years_from_url(url)
    all_songs.extend(songs)
    all_artists.extend(artists)
    all_years.extend(years)

#create df
df = pd.DataFrame({'Song': all_songs, 'Artist': all_artists, 'Year': all_years})
print(df)

                            Song                       Artist   Year
0     "Are You Gonna Be My Girl"                          Jet   2003
1               "Ballroom Blitz"                        Sweet   1973
2               "Black Hole Sun"                  Soundgarden   1994
3               "Blitzkrieg Bop"                      Ramones   1976
4               "Celebrity Skin"                         Hole   1998
..                           ...                          ...    ...
338  "The Trees" (Vault Edition)                         Rush  1970s
339       "What's My Age Again?"                    Blink-182  1990s
340            "Where'd You Go?"  The Mighty Mighty Bosstones  1990s
341      "White Wedding, Part 1"                   Billy Idol  1980s
342                     "Would?"              Alice in Chains  1990s

[343 rows x 3 columns]
